# Drone Trajectory Planner

In this project, we will develop the drone trajectory planner. This notebook serves as the main file for the project, where we will refer to the instructions and demonstrate our code.

Please follow week by week instructions, which includes writing the code in the `src/` folder.

In [3]:
# Import all the files and libraries required for the project
%load_ext autoreload
%autoreload 2
import copy
    
import numpy as np

from src.camera_utils import compute_image_footprint_on_surface, compute_ground_sampling_distance, project_world_point_to_image
from src.data_model import Camera, DatasetSpec
from src.plan_computation import compute_distance_between_images, compute_speed_during_photo_capture, generate_photo_plan_on_grid
from src.visualization import plot_photo_plan

# Week 1: Introduction

No code contribution expected this week

# Week 2: Camera System Modeling and Operations

We plan to
- Model the simple pinhole camera system
- Write utility functions to
    - project a 3D world point to an image
    - Compute image footprint on a surface
    - Compute the Ground Sampling Distance

## Model the camera parameters

We want to model the following camera parameters in Python:
- focal length along x axis (in pixels)
- focal length along y axis (in pixels)
- optical center of the image along the x axis (in pixels)
- optical center of the image along the y axis (in pixels)
- Size of the sensor along the x axis (in mm)
- Size of the sensor along the y axis (in mm)
- Number of pixels in the image along the x axis
- Number of pixels in the image along the y axis

I recommend to use `dataclasses` ([Python documentation](https://docs.python.org/3/library/dataclasses.html), [Blog](https://www.dataquest.io/blog/how-to-use-python-data-classes/) to model these parameters.

$\color{red}{\text{TODO: }}$ Implement `Camera` in `src/data_model.py`

In [5]:
# Define the parameters for Skydio VT300L - Wide camera
# Ref: https://support.skydio.com/hc/en-us/articles/20866347470491-Skydio-X10-camera-and-metadata-overview
fx = 4938.56
fy = 4936.49
cx = 4095.5
cy = 3071.5
sensor_size_x_mm = 13.107 # single pixel size * number of pixels in X dimension
sensor_size_y_mm = 9.830 # single pixel size * number of pixels in Y dimension
image_size_x = 8192
image_size_y = 6144

camera_x10 = Camera(fx, fy, cx, cy, sensor_size_x_mm, sensor_size_y_mm, image_size_x, image_size_y)

In [6]:
print(f"X10 camera model: {camera_x10}")
print(camera_x10.cx, camera_x10.cy)

X10 camera model: Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144)
4095.5 3071.5


## Project 3D world points into the image


![Camera Projection](assets/image_projection.png)
Reference: [Robert Collins CSE483](https://www.cse.psu.edu/~rtc12/CSE486/lecture12.pdf)


Equations to implement:
$$ x = f_x \frac{X}{Z} $$
$$ y = f_y \frac{Y}{Z} $$
$$ u = x + c_x $$
$$ v = y + c_y $$

$\color{red}{\text{TODO: }}$ Implement function `project_world_point_to_image` in `src/camera_utils.py`

In [7]:
point_3d = np.array([25, -30, 50], dtype=np.float32)
expected_uv = np.array([6564.80, 109.60], dtype=np.float32)
uv = project_world_point_to_image(camera_x10, point_3d)
print(f"{point_3d} projected to {uv}")
print(f"Calculated uv: {uv}")
print(f"Expected uv: {expected_uv}")

assert np.allclose(uv, expected_uv, atol=1e-2)

[ 25. -30.  50.] projected to [6564.7803   109.60571]
Calculated uv: [6564.7803   109.60571]
Expected uv: [6564.8  109.6]


## Compute Image Footprint on the surface

We have written code to *project* a 3D point into the image. The reverse operation is reprojection, where we take $(x, y)$ and compute the $(X, Y)$ for a given value of $Z$. Note that while going from 3D to 2D, the depth becomes ambiguous so we need the to specify the $Z$.

An image's footprint is the area on the surface which is captured by the image. We can take the two corners of the image and reproject them at a given distance to obtain the width and length of the image.

$\color{red}{\text{TODO: }}$ Implement function `compute_image_footprint_on_surface` in `src/camera_utils.py`

In [8]:
footprint_at_100m = compute_image_footprint_on_surface(camera_x10, 100)
expected_footprint_at_100m = np.array([165.88, 124.46], dtype=np.float32)

print(f"Footprint at 100m = {footprint_at_100m}")

assert np.allclose(footprint_at_100m, expected_footprint_at_100m, atol=1e-2)


Footprint at 100m = [165.87831271 124.46090238]


In [9]:
footprint_at_200m = compute_image_footprint_on_surface(camera_x10, 200)
expected_footprint_at_200m = expected_footprint_at_100m * 2

print(f"Footprint at 200m = {footprint_at_200m}")

assert np.allclose(footprint_at_200m, expected_footprint_at_200m, atol=1e-2)

Footprint at 200m = [331.75662541 248.92180476]


## Ground Sampling Distance

Ground sampling distance is the length of the ground (in m) captured by a single pixel. We have the image footpring (the dimensions of ground captured by the whole sensor, and the number of pixels along the horizontal and vertical dimension. Can we get GSD from these two quantities?

Note: Please return just one value of the GSD. Take the mininum of the values along the two axes.

In [10]:
gsd_at_100m = compute_ground_sampling_distance(camera_x10, 100)
expected_gsd_at_100m = 0.0202

print(f"GSD at 100m: {gsd_at_100m}")

print(gsd_at_100m)
assert np.allclose(gsd_at_100m, expected_gsd_at_100m, atol=1e-4)

GSD at 100m: 0.020248817469059804
0.020248817469059804


## Bonus: Reprojection from 2D to 3D

If we have a 2d pixel location of a point along with the camera model, can we go back to 3D?
Do we need any additional information.


$\color{red}{\text{TODO: }}$ Implement function `reproject_image_point_to_world` in `src/camera_utils.py` and demonstrate it by running it in the notebook. Confirm that your reprojection + projection function are consistent.

# Week 3: Model the user requirements

For this week, we will model the dataset specifications.

- Overlap: the ratio (in 0 to 1) of scene shared between two consecutive images.
- Sidelap: the ratio (in 0 to 1) of scene shared between two images in adjacent rows.
- Height: the height of the scan above the ground (in meters).
- Scan_dimension_x: the horizontal size of the rectangle to be scanned
- Scan_dimension_y: the vertical size of the rectangle to be scanned
- exposure_time_ms: the exposure time for each image (in milliseconds).


$\color{red}{\text{TODO: }}$ Implement `DatasetSpec` in `src/data_model.py`


In [11]:
# Model the nomimal dataset spec

overlap = 0.7
sidelap = 0.7
height = 30.48 # 100 ft
scan_dimension_x = 150
scan_dimension_y = 150
exposure_time_ms = 2 # 1/500 exposure time

dataset_spec = DatasetSpec(overlap, sidelap, height, scan_dimension_x, scan_dimension_y, exposure_time_ms)

print(f"Nominal specs: {dataset_spec}")
print(dataset_spec.exposure_time_ms)

Nominal specs: DatasetSpec(overlap=0.7, sidelap=0.7, height=30.48, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=2)
2


# Week 4: Compute Distance Between Photos

The overlap and sidelap are the ratio of the dimensions shared between two photos. We already know the footprint of a single image at a given distance. Can we convert the ratio into actual distances? And how does the distance on the surface relate to distance travelled by the camera?

$\color{red}{\text{TODO: }}$ Implement `compute_distance_between_images` in `src/plan_computation.py`



In [12]:
computed_distances = compute_distance_between_images(camera_x10, dataset_spec)
expected_distances = np.array([15.17, 11.38], dtype=np.float32)

print(f"Computed distance for X10 camera with nominal dataset specs: {computed_distances}")

assert np.allclose(computed_distances, expected_distances, atol=1e-2)

Computed distance for X10 camera with nominal dataset specs: [15.16791291 11.38070491]


$\color{red}{\text{TODO: }}$ define more specifications/camera parameters and check the computed distances. Does that align with your expections


In [13]:
camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)

computed_distances_ = compute_distance_between_images(camera_, dataset_spec_)
#print(f"Computed distance: {computed_distances_}")


# Example reducing overlap should increase horizontal distance between images
# Helpful when in scenairos where images require less detail due to flat terrain, or repetitive imagery.
dataset_spec_.overlap = .25 * dataset_spec.overlap
computed_distances_ = compute_distance_between_images(camera_, dataset_spec_)
print(f"Computed distance for original specs: {computed_distances}")
print(f"Computed distance with 25% of .7 overlap: {computed_distances_} \n")


Computed distance for original specs: [15.16791291 11.38070491]
Computed distance with 25% of .7 overlap: [41.71176051 11.38070491] 



In [14]:
# reset
camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)
# Example increaseing overlap should decrease vertical distance between images
# Helpful in scenarios where images require high detail due to complex terrain, imagery.
dataset_spec_.sidelap = .9
computed_distances_ = compute_distance_between_images(camera_, dataset_spec_)
print(f"Computed distance for original specs: {computed_distances}")
print(f"Computed distance .9 sidelap: {computed_distances_} \n")

Computed distance for original specs: [15.16791291 11.38070491]
Computed distance .9 sidelap: [15.16791291  3.7935683 ] 



In [15]:
# Example increase drone height in data spec to 50 feet. 
# This would be useful in scenarios where the drone is flying over a large area and needs less detail.
# This should increase the footprints thus increaing both vertical and horizontal overlaps.
# This should also increase ground sampling distance.
dataset_spec_.height = 50.8 # 50 feet
computed_distances_ = compute_distance_between_images(camera_, dataset_spec_)
ground_sampling_0 = compute_ground_sampling_distance(camera_x10, dataset_spec.height)
ground_sampling_1 = compute_ground_sampling_distance(camera_, dataset_spec_.height)

print(f"Computed distance for original specs: {computed_distances}")
print(f"Computed distance with 50 feet height: {computed_distances_} \n")

print(f"Ground sampling distance for original specs: {ground_sampling_0}")
print(f"Ground sampling distance with 50 feet height: {ground_sampling_1}\n")

Computed distance for original specs: [15.16791291 11.38070491]
Computed distance with 50 feet height: [25.27985486  6.32261384] 

Ground sampling distance for original specs: 0.006171839564569429
Ground sampling distance with 50 feet height: 0.01028639927428238



In [16]:
# reset
camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)
# Example increase image size horizontally and veritcally 
# This would be useful in scenarios where more detial is required in the images from a higher resolution camera.
# This should increase the footprints thus increaing both vertical and horizontal overlaps distances. 
camera_.image_size_x_px = 8192 * 2
camera_.image_size_y_px = 6144 * 2
computed_distances_ = compute_distance_between_images(camera_, dataset_spec_)
print(f"Computed distance for original specs: {computed_distances}")
print(f"Computed distance with double image size x and y: {computed_distances_} \n")

Computed distance for original specs: [15.16791291 11.38070491]
Computed distance with double image size x and y: [30.33582583 22.76140983] 



In [17]:
# reset
camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)
# Example decrease focal length 
# This would useful in covering an area with less images or less overlaps.
# This should increase the footprints thus increasing both vertical and horizontal overlaps distances. 
camera_.fx = camera_.fx * .5
camera_.fy = camera_.fy * .5
computed_distances_ = compute_distance_between_images(camera_, dataset_spec_)

print(f"Computed distance for original specs: {computed_distances}")
print(f"Computed distance with double focal lentghs both x and y: {computed_distances_}\n")

Computed distance for original specs: [15.16791291 11.38070491]
Computed distance with double focal lentghs both x and y: [30.33582583 22.76140983]



## Bonus: Non-Nadir photos

We have solved for the distance assuming that the camera is facing straight down to the ground. This is called [Nadir scanning](https://support.esri.com/en-us/gis-dictionary/nadir). However, in practise we might want a custom gimbal angle.

Your bonus task is to make the distance computation general. Introduce a double `camera_angle` parameter (which is the angle from the X-axis) in the dataset specification, and work out how to adapt your computation. Feel free to reach out to Ayush to discuss ideas and assumptions!

![Non Nadir Footprint](assets/non_nadir_gimbal_angle.png)

# Week 5: Compute Maximum Speed For Blur Free Photos

To restrict motion blur due to camera movement to tolerable limits, we need to restrict the speed such that the image contents move less than 1px away. 

How much does 1px of movement translate to movement of the scene on the ground? It is the ground sampling distance!
From previous week, we know that this is the maximum movement the camera can have. 
We have the distance now. To get speed we need to divide it with time. Do we have time already in our data models?

$\color{red}{\text{TODO: }}$ Implement `compute_speed_during_photo_capture` in `src/plan_computation.py`.

In [18]:
computed_speed = compute_speed_during_photo_capture(camera_x10, dataset_spec, allowed_movement_px=1)
expected_speed = 3.09

print(f"Computed speed during photo captures: {computed_speed:.2f}")

assert np.allclose(computed_speed, expected_speed, atol=1e-2)

Computed speed during photo captures: 3.09


$\color{red}{\text{TODO: }}$ define more specifications/camera parameters and check the computed distances. Does that align with your expections


In [19]:
camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)

# Example increasing exposure time for low light scenarios 
# This should decrease the speed required to shoot blur free images.
dataset_spec_.exposure_time_ms = 10
computed_speed_ = compute_speed_during_photo_capture(camera_, dataset_spec_)
print(f"Base line compute speed: {computed_speed:.2f}")
print(f"Computed speed at higher exposure: {computed_speed_:.2f}")

Base line compute speed: 3.09
Computed speed at higher exposure: 0.62


In [20]:
camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)

# Example decreasing ground sampling distance by increasing focal length to shoot more detailed images
# This should decrease the speed required to shoot blur free images.
camera_.fx = camera_.fx * 2
camera_.fy = camera_.fy * 2
computed_speed_ = compute_speed_during_photo_capture(camera_, dataset_spec_)
print(f"Base line compute speed: {computed_speed:.2f}")
print(f"Computed speed at higher focal length: {computed_speed_:.2f}")

Base line compute speed: 3.09
Computed speed at higher focal length: 1.54


In [21]:
camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)

# Example increasing allowed movement in pixels for faster flights
# This could be helpful in scenarios where rapid surevy of large areas for large items required.
# This should increase speed.
computed_speed_ = compute_speed_during_photo_capture(camera_, dataset_spec_, 2)
print(f"Base line compute speed: {computed_speed:.2f}")
print(f"Computed speed at increased allowed pixel movement: {computed_speed_:.2f}")

Base line compute speed: 3.09
Computed speed at increased allowed pixel movement: 6.17


# Week 6: Generate Full Flight Plans  

We now have all the tools to generate the full flight plan.

Steps for this week:
1. Define the `Waypoint` data model. What attributes should the data model have?
   1. For Nadir scans, just the position of the camera is enough as we will always look drown to the ground.
   2. For general case (bonus), we also need to define where the drone will look at.
3. Implement the function `generate_photo_plan_on_grid` to generate the full plan.
   1. Compute the maximum distance between two images, horizontally and vertically.
   2. Layer the images such that we cover the whole scan area. Note that you need to take care when the scan dimension is not a multiple of distance between images. Example: to cover 45m length with 10m between images, we would need 4.5 images. Not possible. 4 images would not satisfy the overlap, so we should go with 5. How should we arrange 5 images in the given 45m.
   3. Assign the speed to each waypoint.

$\color{red}{\text{TODO: }}$ Implement:
- `Waypoint` in `src/data_model.py`
- `generate_photo_plan_on_grid` in `src/plan_computation.py`.

In [101]:
computed_plan = generate_photo_plan_on_grid(camera_x10, dataset_spec) 

print(f"Computed plan with {len(computed_plan)} waypoints")

Computed plan with 165 waypoints


In [102]:
MAX_NUM_WAYPOINTS_TO_PRINT = 20

for idx, waypoint in enumerate(computed_plan[:20]):
    print(f"Idx {idx}: {waypoint}")
if len(computed_plan) >= MAX_NUM_WAYPOINTS_TO_PRINT:
    print("...")

Idx 0: Waypoint(x=0.0, y=0.0, speed=3.0859197822847144)
Idx 1: Waypoint(x=15.0, y=0.0, speed=3.0859197822847144)
Idx 2: Waypoint(x=30.0, y=0.0, speed=3.0859197822847144)
Idx 3: Waypoint(x=45.0, y=0.0, speed=3.0859197822847144)
Idx 4: Waypoint(x=60.0, y=0.0, speed=3.0859197822847144)
Idx 5: Waypoint(x=75.0, y=0.0, speed=3.0859197822847144)
Idx 6: Waypoint(x=90.0, y=0.0, speed=3.0859197822847144)
Idx 7: Waypoint(x=105.0, y=0.0, speed=3.0859197822847144)
Idx 8: Waypoint(x=120.0, y=0.0, speed=3.0859197822847144)
Idx 9: Waypoint(x=135.0, y=0.0, speed=3.0859197822847144)
Idx 10: Waypoint(x=150.0, y=0.0, speed=3.0859197822847144)
Idx 11: Waypoint(x=150.0, y=10.714285714285714, speed=3.0859197822847144)
Idx 12: Waypoint(x=135.0, y=10.714285714285714, speed=3.0859197822847144)
Idx 13: Waypoint(x=120.0, y=10.714285714285714, speed=3.0859197822847144)
Idx 14: Waypoint(x=105.0, y=10.714285714285714, speed=3.0859197822847144)
Idx 15: Waypoint(x=90.0, y=10.714285714285714, speed=3.0859197822847144)


## Bonus: Time computation 

if you have some time, you can implement a time computation function. We can make the drone fly as fast as possible between photos, but make sure it can decelerate back to the required speed at the photos. Please use the following data: 
- Max drone speed: 16m/s.
- Max acceleration: 3.5 m/s^2.

Hint: you might need to use a trapezoidal speed profile

# Week 7: Visualize Flight Plans

This week, we will use a third party plotting framework called [Plotly](https://plotly.com/python/) to visualize our plans. Please follow this [tutorial](https://www.kaggle.com/code/kanncaa1/plotly-tutorial-for-beginners) to gain some basic experience with Plotly, and then come up with your own visualization function. You are free to choose to come up with your own visualization, and use something other than Plotly.

$\color{red}{\text{TODO: }}$ Implement `plot_photo_plan` in `src/visualization.py`

In [264]:
fig = plot_photo_plan(computed_plan)
fig.show()

$\color{red}{\text{TODO: }}$ Compute the following ablations (and any other you can think of). 
You need to describe the input params you are changing, what impact you can observe, explanation behind the change in output, and practical implication of the correlation.

1. Change overlap and confirm it affects the consecutive images
2. Change sidelap and confirm it does not affect the consecutive images
3. Change the height of the scan and document the affect on scan plans
4. Change exposure time

In [265]:
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)

dataset_spec_.exposure_time_ms = 1000

print(compute_speed_during_photo_capture(camera_,dataset_spec_))
print(camera_, dataset_spec_)

fig = plot_photo_plan(generate_photo_plan_on_grid(camera_, dataset_spec_))
fig.show()

0.006171839564569429
Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144) DatasetSpec(overlap=0.7, sidelap=0.7, height=30.48, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=1000)


$\color{red}{\text{Experiment 1 }}$ Change overlap and confirm it affects the consecutive images

* **Parameter Change:** Overlap from .7 to 0  
<br>
* **Expected Outcome:**  Reduction of the numer of consecutive images  
<br>
* **Explanation:** Overlap has a direct relationship with consecutive photo numbers. If overlap increases, images increase, if overlap reduces the number of images reduces.  
<br> 
* **Math:** 
In `compute_distance_between_images` the overlap ratio is a determining factor in the distance between consecutive images.    
<br>
`horizontal_overlap = (1 - dataset_spec.overlap) * footprint_x`  <br><br>This product is then further used to plan flight path by:
<br>
 `generate_photo_plan_on_grid` `images_x_axis = math.ceil(dataset_spec.scan_dimension_x / max_distance[0])`
 <br><br>Where the max distance determines how many points the drone needs to stop to take an image.

* **Use Case:** Reducing overlap will generally lead to loss of details, in most cases increasing overlap is the best choice for image quality.

In [266]:
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)

dataset_spec_.overlap = 0
#dataset_spec_.sidelap = 0


print(camera_, dataset_spec_)

fig = plot_photo_plan(generate_photo_plan_on_grid(camera_, dataset_spec_))
fig.show()

Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144) DatasetSpec(overlap=0, sidelap=0.7, height=30.48, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=2)


$\color{red}{\text{Experiment 2 }}$ Change sidelap and confirm it does not affect the consecutive images

* **Parameter Change:** Sidelap from .7 to 0  
<br>
* **Expected Outcome:**  No effect on the numer of consecutive images. However there will be a reduction in the amount of rows.  
<br>
* **Explanation:** Sidelap has a direct relationship with number of rows of photos. If sidelap increases, number of rows increase increase, if sidelap reduces the number of images reduces.  
<br> 
* **Math:** 
In `compute_distance_between_images` the sidelap ratio is a determining factor in the number of rows.    
<br>
`vertical_overlap = (1 - dataset_spec.sidelap) * footprint_y`  <br><br>This product is then further used to plan flight path by:
<br>
 `generate_photo_plan_on_grid` `images_y_axis = math.ceil(dataset_spec.scan_dimension_y / max_distance[1])`
 <br><br>Where the max distance determines how many rows of consecutive points the drone needs to stop to take an image.

* **Use Case:** Reducing overlap will generally lead to loss of details, in most cases increasing overlap is the best choice for image quality.

In [271]:
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)


dataset_spec_.sidelap = 0


print(camera_, dataset_spec_)

fig = plot_photo_plan(generate_photo_plan_on_grid(camera_, dataset_spec_))
fig.show()

Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144) DatasetSpec(overlap=0.7, sidelap=0, height=30.48, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=2)


$\color{red}{\text{Experiment 3 }}$ Change the height of the scan and document the affect on scan plans

* **Parameter Change:** Two models for reduced height at 10m and incresed height at 100m  
<br>
* **Expected Outcome:**  Lower height increases both the number of rows, consecutive images and reduces the max speed of the drone. Increased heights will result in less rows and consecutive images while increasing the speed at which the drone can operate. This also increases the ground sampling distance. This assumes a constant square area. 
<br>
* **Explanation:** As the height of the drone changes so does the ground sampling distance and the footprints in both the x and y direction. For example at a higher elevation the footprint is larger requiring less images to be taken to cover a given area. 
<br>
    Speed also changes as ground sampling distance increases resulting in more distance being cover over a given period of time. 
<br> 
* **Math:** 
In `generate_photo_plan_on_grid` the number of images per axis is directly affected by height increase or decreases.    
<br>
    * `images_x_axis = math.ceil(dataset_spec.scan_dimension_x / max_distance[0])`
<br>
    * `images_y_axis = math.ceil(dataset_spec.scan_dimension_y  / max_distance[1])`  
    <br>
The equations underneath this can be simply definied for an axis by:
<br>
        * $$\text{number of images on axis} = \frac {\text{image dimension axis}} {(\text{overlap axis} * (\text{scan dimension axis} * \text{height}))}$$  
    * An equation for speed = distance /time can be broken down into:
        * $$ \frac {\text{distance from surface}}{\text{max focal length }} \times \frac{1}{\text{time}}$$

* **Use Case:** Reducing the height along may be useful for taking more detailed pictures by way of the decreased ground sampling. Increasing height may be useful for taking large areas of images quiclky where detial quality is not important, however generally if height is increased then focal length needs to be increased as well to maintin quality.

In [274]:
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)


dataset_spec_.height = 10


print(camera_, dataset_spec_)
print("Original Speed", compute_speed_during_photo_capture(camera_x10,dataset_spec))
print("Reduced speed: ", compute_speed_during_photo_capture(camera_,dataset_spec_))
print("Original Footprint", compute_image_footprint_on_surface(camera_x10, dataset_spec.height))
print("Reduced Footprint", compute_image_footprint_on_surface(camera_, dataset_spec_.height))
print("Original GSD", compute_ground_sampling_distance(camera_x10, dataset_spec.height))
print("Reduced GSD", compute_ground_sampling_distance(camera_, dataset_spec_.height))
print("Original Distance between images", compute_distance_between_images(camera_x10, dataset_spec))
print("Reduced Distance between images", compute_distance_between_images(camera_, dataset_spec_))

fig = plot_photo_plan(generate_photo_plan_on_grid(camera_, dataset_spec_))
fig.show()

Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144) DatasetSpec(overlap=0.7, sidelap=0.7, height=10, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=2)
Original Speed 3.0859197822847144
Reduced speed:  1.0124408734529904
Original Footprint [50.55970971 37.93568305]
Reduced Footprint [16.58783127 12.44609024]
Original GSD 0.006171839564569429
Reduced GSD 0.0020248817469059807
Original Distance between images [15.16791291 11.38070491]
Reduced Distance between images [4.97634938 3.73382707]


In [273]:
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)


dataset_spec_.height = 100


print(camera_, dataset_spec_)
print("Original Speed", compute_speed_during_photo_capture(camera_x10,dataset_spec))
print("Increased speed: ", compute_speed_during_photo_capture(camera_,dataset_spec_))
print("Original Footprint", compute_image_footprint_on_surface(camera_x10, dataset_spec.height))
print("Increased Footprint", compute_image_footprint_on_surface(camera_, dataset_spec_.height))
print("Original GSD", compute_ground_sampling_distance(camera_x10, dataset_spec.height))
print("Increased GSD", compute_ground_sampling_distance(camera_, dataset_spec_.height))
print("Original Distance between images", compute_distance_between_images(camera_x10, dataset_spec))
print("Increased Distance between images", compute_distance_between_images(camera_, dataset_spec_))

fig = plot_photo_plan(generate_photo_plan_on_grid(camera_, dataset_spec_))
fig.show()

Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144) DatasetSpec(overlap=0.7, sidelap=0.7, height=100, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=2)
Original Speed 3.0859197822847144
Increased speed:  10.1244087345299
Original Footprint [50.55970971 37.93568305]
Increased Footprint [165.87831271 124.46090238]
Original GSD 0.006171839564569429
Increased GSD 0.020248817469059804
Original Distance between images [15.16791291 11.38070491]
Increased Distance between images [49.76349381 37.33827071]
